In [5]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

np.random.seed(7)

In [6]:
top_words=100

#(X_train,y_train),(X_test,y_test)=imdb.load_data(num_words=top_words,)

np_load_old = np.load
# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=top_words)

# restore np.load for future normal usage
np.load = np_load_old

In [8]:
X_train=train_data
y_train=train_labels
X_test=test_data
y_test=test_labels

In [9]:
print(X_train[1]) # value will be less less than 100 we have limited the indexes
print(type(X_train[1]))
print(len(X_train[1]))

[1, 2, 2, 2, 2, 78, 2, 5, 6, 2, 2, 2, 2, 26, 4, 2, 8, 2, 2, 14, 2, 20, 13, 2, 2, 2, 2, 5, 2, 2, 2, 21, 14, 69, 2, 8, 30, 23, 7, 4, 2, 2, 93, 4, 2, 9, 2, 2, 5, 2, 4, 2, 9, 35, 2, 4, 2, 9, 2, 2, 4, 2, 9, 4, 2, 2, 19, 4, 2, 5, 89, 29, 2, 46, 37, 4, 2, 9, 45, 43, 38, 2, 2, 2, 4, 2, 26, 2, 5, 2, 11, 2, 2, 4, 2, 9, 2, 2, 7, 2, 2, 2, 2, 2, 2, 2, 2, 15, 2, 2, 68, 2, 2, 15, 2, 2, 2, 98, 5, 4, 2, 9, 43, 2, 2, 15, 2, 2, 5, 2, 2, 11, 2, 2, 2, 50, 9, 2, 2, 2, 5, 2, 2, 2, 2, 5, 4, 2, 2, 2, 2, 18, 2, 32, 2, 2, 14, 9, 6, 2, 78, 22, 2, 64, 2, 9, 8, 2, 2, 23, 4, 2, 15, 16, 4, 2, 5, 28, 6, 52, 2, 2, 33, 89, 78, 2, 16, 2, 95]
<class 'list'>
189


In [10]:
# pad input sequences to have all reviews of equal length

max_length_review=300

X_train=sequence.pad_sequences(X_train,max_length_review) # length of X_train will be 300 now not 189

X_test=sequence.pad_sequences(X_test,max_length_review)


print(X_train.shape)
print(X_train[1])


(25000, 300)
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  2  2  2  2 78  2  5  6
  2  2  2  2 26  4  2  8  2  2 14  2 20 13  2  2  2  2  5  2  2  2 21 14
 69  2  8 30 23  7  4  2  2 93  4  2  9  2  2  5  2  4  2  9 35  2  4  2
  9  2  2  4  2  9  4  2  2 19  4  2  5 89 29  2 46 37  4  2  9 45 43 38
  2  2  2  4  2 26  2  5  2 11  2  2  4  2  9  2  2  7  2  2  2  2  2  2
  2  2 15  2  2 68  2  2 15  2  2  2 98  5  4  2  9 43  2  2 15  2  2  5
  2  2 11  2  2  2 50  9  2  2  2  5  2  2  2  2  5  4  2  2  2  2 18  2
 32  2  2 14  9  6  2 78 22  2 64  2  9  8  2  2 23  4  2 15 16  4  2  5
 28  6 52  2  2 33 89 78  2 16  2 95]


# Outlining Model

In [13]:
embedding_vector_length=32

model=Sequential()

# embedding layer is ust like word2vec.Reason for not using word2vec is that it's not learned as a part of model.

#Embedding layers is learned as a part of model

model.add(Embedding(top_words+1,embedding_vector_length,input_length=max_length_review))

model.add(LSTM(100))

model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 32)           3232      
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 56,533
Trainable params: 56,533
Non-trainable params: 0
_________________________________________________________________
None


# Train

In [16]:
model.fit(X_train,y_train,epochs=4,batch_size=64)

scores=model.evaluate(X_test,y_test,verbose=0)
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
#print('Accuracy : %.2f%%' % ((scores[1]*100))

C:\Users\S BL\Anaconda3\lib\site-packages\keras\models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/4
25000/25000 [==============================] - 409s 16ms/step - loss: 0.6467 - acc: 0.6196
Epoch 2/4
25000/25000 [==============================] - 475s 19ms/step - loss: 0.6020 - acc: 0.6768
Epoch 3/4
25000/25000 [==============================] - 387s 15ms/step - loss: 0.5843 - acc: 0.6943
Epoch 4/4
25000/25000 [==============================] - 370s 15ms/step - loss: 0.5663 - acc: 0.7098
Accuracy: 72.92%
